In [1]:
from CNN.CNN import MultivariateCNN
from models_utils.Datasets import *
from models_utils.GLOBALS import *

from RF_XGB.XGBoost import train_xgb
from CNN.cnn_utils import train_cnn, get_train_data
from models_utils.utils import get_results

In [2]:
# get train_data
train_data = pd.read_csv('csv/secret_all_data_features.csv', index_col=0) 
train_data['activity'] = train_data['activity'].map(activity_id_mapping)
data_type_1 = train_data[train_data['sensor'] == 'vicon'].reset_index().rename(columns={'sample_id': 'id'})
data_type_2 = train_data[train_data['sensor'] == 'smartwatch'].reset_index().rename(columns={'sample_id': 'id'})

In [3]:
# sizes of padding/cutting
target_size_type1 = 4000
target_size_type2 = 1300

In [4]:
# train or load models
train_or_load_models = 'train'
if train_or_load_models == 'train':
    model_CNN_type1 = train_cnn('Type1CNNtoLSTMModel', data_type_1, '1', target_size_type1, 30, 64, 0.001)
    model_CNN_type2 = train_cnn('Type2CNNtoLSTMModel', data_type_2, '2', target_size_type2, 20, 64, 0.001)
elif train_or_load_models == 'load':
    model_CNN_type1 = MultivariateCNN(3, target_size_type1, 18).to(device)
    model_CNN_type1.load_state_dict(torch.load('Type1CNNtoLSTMModel.pth'))
    model_CNN_type2 = MultivariateCNN(3, target_size_type2, 18).to(device)
    model_CNN_type2.load_state_dict(torch.load(f'Type2CNNtoLSTMModel.pth'))
else:
    raise ValueError('Wrong train or load')

--------------
Saving Best Model with loss: 1.701795762235468
Epoch [1/30], Epoch Duration: 102.82 seconds
Epoch [1/30], Training Loss: 2.1252, Training Accuracy: 22.80%
Epoch [1/30], Validation Loss: 1.7018, Validation Accuracy: 28.23%
--------------
Saving Best Model with loss: 1.4520286619663239
Epoch [2/30], Epoch Duration: 29.49 seconds
Epoch [2/30], Training Loss: 1.5334, Training Accuracy: 37.70%
Epoch [2/30], Validation Loss: 1.4520, Validation Accuracy: 42.30%
--------------
Saving Best Model with loss: 1.282708690925078
Epoch [3/30], Epoch Duration: 30.34 seconds
Epoch [3/30], Training Loss: 1.2239, Training Accuracy: 53.04%
Epoch [3/30], Validation Loss: 1.2827, Validation Accuracy: 49.63%
--------------
Saving Best Model with loss: 1.0688610158183358
Epoch [4/30], Epoch Duration: 30.19 seconds
Epoch [4/30], Training Loss: 0.8869, Training Accuracy: 67.80%
Epoch [4/30], Validation Loss: 1.0689, Validation Accuracy: 60.50%
--------------
Saving Best Model with loss: 0.9462952

In [ ]:
# calculate or load train_data
embedding_names = [f'embedding_feature_{i + 1}' for i in range(18)]
calculate_or_load_train_data = 'load'
if calculate_or_load_train_data == 'calculate':
    data_type_1, data_type_2 = get_train_data('train_CNN_to_LSTM_data.csv', model_CNN_type1, model_CNN_type2,
                                              target_size_type1, target_size_type2, embedding_size=18,
                                              is_autoencoder=False)
elif calculate_or_load_train_data == 'load':
    data_type_1 = pd.read_csv('train_CNN_to_LSTM_data_type1.csv')
    data_type_2 = pd.read_csv('train_CNN_to_LSTM_data_type2.csv')

In [ ]:
# train xgb
cols_to_drop_type_1 = ['activity']
xgb_classifier_type1, label_encoder_type1 = train_xgb(data_type_1, cols_to_drop_type_1, n_estimators=200)
cols_to_drop_type_2 = ['activity', 'dominant_freq_y', 'count_x', 'count_y', 'count_z', 'dominant_freq_x']
xgb_classifier_type2, label_encoder_type2 = train_xgb(data_type_2, cols_to_drop_type_2, n_estimators=200)

In [ ]:
# get and save results for test data
results = get_results(model_CNN_type1, model_CNN_type2, xgb_classifier_type1, xgb_classifier_type2, label_encoder_type1,
                      label_encoder_type2, target_size_type1, target_size_type2, cols_to_drop_type_1,
                      cols_to_drop_type_2, embedding_names)
results.to_csv('cnn_to_lstm_results.csv')